# CONVERTENDO AS CURVAS (CANAIS) DOS ARQUIVOS DLIS EM DATAFRAMES

### IMPORTANDO AS BIBLIOTECAS NECESSÁRIAS

In [1]:
import glob
from dlisio import dlis
import pandas as pd
import re
from collections import Iterable
import numpy as np

C:\Users\Dell\AppData\Local\Temp\ipykernel_952\2946528182.py:5: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3, and in 3.9 it will stop working
  from collections import Iterable


In [2]:
import warnings
warnings.filterwarnings("ignore")

### MAIS A SOBRE DLISIO: BIBLIOTECA PARA LEITURA E ANÁLISE DE DADOS DOS ARQUIVOS .DLIS:

https://dlisio.readthedocs.io/en/latest/

### FUNÇÕES UTILIZADAS

#### FUNÇÃO PARA REMOVER VALORES REPETIDOS DE UMA LISTA

In [34]:
def remove_repetidos(lista):
    
    '''INPUT: 
    - "lista": lista que contém elementos repetidos.
    
    OUTPUTs:
    - Lista sem elementos repetidos. '''
    
    l = []
    for i in lista:
        if i not in l:
            l.append(i)
    l.sort()
    return l

#### FUNÇÃO PARA DESENCADEAR LISTAS

In [59]:
def flatten(lis):
    
    '''INPUT: 
    - "lis": lista que é uma "lista de listas".
    
    OUTPUT:
    - Uma lista simples. '''
    
    for item in lis:
        if isinstance(item, Iterable) and not isinstance(item, str):
            for x in flatten(item):
                yield x
        else:        
            yield item

#### FUNÇÃO QUE CRIA UM DATAFRAME CONTENDO PARÂMETROS DOS DLIS

In [36]:
def summary_dataframe(object, **kwargs):
    
    '''INPUTS:  
    - "object": arquivo_lógico.object, onde object, no caso, será "Channel". 
    - "kwargs": atributos do objeto, escolhidos para compor o dataframe.
    
    OUTPUT:
    - Dataframe contendo os canais e seus atributos.
    
    Para saber mais sobre objetos e atributos da biblioteca Dlisio: https://dlisio.readthedocs.io/en/latest/dlis/api.html#basic-object '''

    # Create an empty dataframe
    df = pd.DataFrame()
    
    # Iterate over each of the keyword arguments
    for i, (key, value) in enumerate(kwargs.items()):
        list_of_values = []
        
        # Iterate over each parameter and get the relevant key
        for item in object:
            # Account for any missing values.
            try:
                x = getattr(item, key)
                list_of_values.append(x)
            except:
                list_of_values.append('')
                continue
        
        # Add a new column to our data frame
        df[value]=list_of_values
    
    # Sort the dataframe by column 1 and return it
    return df.sort_values(df.columns[0])

#### FUNÇÃO PARA FILTRAR A PALAVRA ESCOLHIDA (PARÂMETRO FÍSICO) E RETORNAR UMA LISTA COM O NOME DOS CANAIS DO RESPECTIVO PARÂMETRO

In [70]:
def filtro(summary_df, nome_parametro):
    
    '''INPUTS:
    - "summary_df": dataframe gerado com a função summary_dataframe.
    - "nome_parametro": nome ou parte do nome do parâmetro da curva. Ex.: "Gamma", "density". 
    
    OUTPUT:
    - Lista contendo o nome dos canais que contém o parâmetro físico escolhido. '''
    
    df_novo = summary_df[summary_df['Long Name'].str.contains(nome_parametro)]
    lista = df_novo['Name'].to_list()
    return lista

#### FUNÇÃO PARA ACOPLAR A LITOLOGIA NO DATAFRAME DE CURVAS

In [99]:
def acoplador(dataframe_curvas, dataframe_litologias):
    
    '''INPUTS:
    - "dataframe_curvas": dataframe gerado com as curvas selecionadas.
    - "dataframe_litologias": dataframe gerado a partir do arquivo AGP.
    
    OUTPUT:
    - Dataframe das curvas com uma coluna contendo o nome da litologia e/ou uma coluna contendo o código da litologia
    de acordo com a profundidade. '''
    
    dataframe_litologias['Profundidade'] = dataframe_litologias['Profundidade'].astype(float)
    dataframe_curvas['lito'] = None
    dataframe_curvas['codigo_lito'] = np.nan
    curvas_min = dataframe_curvas[dataframe_curvas['index'] >= dataframe_litologias['Profundidade'].min()]
    
    if ('Code' in dataframe_litologias.columns) and ('Rock' in dataframe_litologias.columns):
        print('O DATAFRAME POSSUI AS COLUNAS CODE E ROCK')
        for i in dataframe_litologias.index:
            for j in dataframe_curvas['index']:
                try:
                    
                    if dataframe_litologias['Profundidade'][i] <= j < dataframe_litologias['Profundidade'][i+1]:
                        dataframe_curvas.loc[dataframe_curvas['index'] == j, 'lito'] = dataframe_litologias.loc[dataframe_litologias.index == i, 'Rock'].values[0]                   
                        dataframe_curvas.loc[dataframe_curvas['index'] == j, 'codigo_lito'] = dataframe_litologias.loc[dataframe_litologias.index == i, 'Code'].values[0]
               
                except Exception as e:
                    print(f'erro do code e rock {e}')
                    pass
        
    elif ('Rock' in dataframe_litologias.columns) and ('Code' not in dataframe_litologias.columns):
        print('O DATAFRAME POSSUI SOMENTE A COLUNA ROCK')
        for i in dataframe_litologias.index:
            for j in dataframe_curvas['index']:
                try:
                    
                    if dataframe_litologias['Profundidade'][i] <= j < dataframe_litologias['Profundidade'][i+1]:
                         dataframe_curvas.loc[dataframe_curvas['index'] == j, 'lito'] = dataframe_litologias.loc[dataframe_litologias.index == i, 'Rock'].values[0]
                   
                except Exception as e:
                    print(f'erro do rock {e}')
                    pass
                
                
    elif ('Code' in dataframe_litologias.columns) and  ('Rock' not in dataframe_litologias.columns):
        print('O DATAFRAME POSSUI SOMENTE A COLUNA CODE')
        for i in dataframe_litologias.index:
            for j in dataframe_curvas['index']:
                try:
                    
                    if dataframe_litologias['Profundidade'][i] <= j < dataframe_litologias['Profundidade'][i+1]:
                        dataframe_curvas.loc[dataframe_curvas['index'] == j, 'codigo_lito'] = dataframe_litologias.loc[dataframe_litologias.index == i, 'Code'].values[0]
           
                except Exception as e:
                    print(f'erro do code {e}')
                    pass
                
    return dataframe_curvas

### VARIÁVEIS ENCONTRADAS NO NOTEBOOK

**dicio_dlis:** dicionário que contém todos os arquivos dlis da pasta selecionada

**lista_logicos1:** lista encadeada de todos os lógicos dos arquivos dlis

**lista_logicos2:** lista de todos os lógicos dos arquivos dlis

**lista_frames:** lista de todos os frames dos dlis

**lista_frames2:** lista sem nomes repetidos de todos os frames dos dlis

**lista_sumario:** lista contendo os dataframes com os parâmetros dos dlis

**channels:** dataframe com os parâmetros dos dlis

**propriedades:** lista onde o usuário coloca as propriedades físicas das curvas do filtro

**lista_alvo:** lista com o nome dos canais filtrados

**profundidade:** lista onde o usuário coloca uma string do Long Name da profundidade

**lista_profundidade:** lista com o nome dos canais de profundidade filtrados a partir da string anterior

**lista_curvas:** lista de dataframes, onde cada dataframe é uma curva (canal)

**lista_curvas2:** lista com os mesmos dataframes, mas sem os valores constantes

**curvas_alvo:** dataframe contendo todas as curvas com a profundidade indexada

**curvas_sem_indice:** dataframe contendo todas as curvas com a profundidade como coluna 


### LENDO OS ARQUIVOS DLIS

In [51]:
#colocar o nome da pasta, em formato de string, onde os arquivos dlis se encontram
path = 'test'

In [52]:
lista_dlis = []
lista_read = []

#importando todos os arquivos dlis da pasta '
for v, path_dlis in enumerate(glob.glob(f'{path}**/*dlis')):
    lista_dlis.append(f'dlis_{v}')
    lista_read.append(path_dlis)

#dicionário com todos os arquivos dlis que se encontram na pasta
dicio_dlis = dict(zip(lista_dlis, lista_read))

In [53]:
#lendo os arquivos dlis com a biblioteca dlisio
lista_logicos1 = []
for i, v in dicio_dlis.items():
    f, *tail = dlis.load(dicio_dlis[i])
    tail.append(f)
    #colocando todos os arquivos lógicos na lista: lista_logicos1 
    lista_logicos1.append(tail)

In [60]:
#utilizando a função para desencadear a lista
lista_logicos2 = list(flatten(lista_logicos1))

In [61]:
lista_logicos2

[LogicalFile(1APS 0029  APBRA1APS 0029  AP2),
 LogicalFile(1APS 0029  APBRA1APS 0029  AP3),
 LogicalFile(1APS 0029  APBRA1APS 0029  AP4),
 LogicalFile(1APS 0029  APBRA1APS 0029  AP5),
 LogicalFile(1APS 0029  APBRA1APS 0029  AP6),
 LogicalFile(1APS 0029  APBRA1APS 0029  AP7),
 LogicalFile(1APS 0029  APBRA1APS 0029  AP8),
 LogicalFile(1APS 0029  APBRA1APS 0029  AP9),
 LogicalFile(1APS 0029  APBRA1APS 0029  AP10),
 LogicalFile(1APS 0029  APBRA1APS 0029  AP11),
 LogicalFile(1APS 0029  APBRA1APS 0029  AP12),
 LogicalFile(1APS 0029  APBRA1APS 0029  AP13),
 LogicalFile(1APS 0029  APBRA1APS 0029  AP14),
 LogicalFile(1APS 0029  APBRA1APS 0029  AP15),
 LogicalFile(1APS 0029  APBRA1APS 0029  AP16),
 LogicalFile(1APS 0029  APBRA1APS 0029  AP17),
 LogicalFile(1APS 0029  APBRA1APS 0029  AP18),
 LogicalFile(1APS 0029  APBRA1APS 0029  AP19),
 LogicalFile(1APS 0029  APBRA1APS 0029  AP20),
 LogicalFile(1APS 0029  APBRA1APS 0029  AP21),
 LogicalFile(1APS 0029  APBRA1APS 0029  AP22),
 LogicalFile(1APS 002

### EXIBINDO O CABEÇALHO DOS 4 PRIMEIROS ARQUIVOS LÓGICOS DA LISTA_LÓGICOS2

In [ ]:
origin, *origin_tail = lista_logicos2[0].origins
#ver de qual dlis são esses logicos

In [ ]:
origin.describe()

------
Origin
------
name   : DEFINING-ORIGIN
origin : 0
copy   : 0

Logical file ID          : 1APS 0029  APBRA1APS 0029  AP2
File set name and number : SPEP-AP-01C1 / 37990
File number and type     : 169414237 / RECALL-DATA

Field                   : LIS_2000/11/11:17:53:
Well (id/name)          : 1APS 0029  AP / 1APS 0029  AP
Produced by (code/name) : 150 /
Produced for            : PETROBRAS
Created                 : 2019-11-11 17:04:41

Created by              : PETROBANK, (version: 4)
Other programs/services : XFER_DLWRIT 1.00


In [ ]:
origin2, *origin_tail2 = lista_logicos2[1].origins
origin2.describe()

------
Origin
------
name   : DEFINING-ORIGIN
origin : 0
copy   : 0

Logical file ID          : 1APS 0029  APBRA1APS 0029  AP3
File set name and number : SPEP-AP-01C1 / 37990
File number and type     : 169414237 / RECALL-DATA

Field                   : LIS_2000/11/11:17:53:
Well (id/name)          : 1APS 0029  AP / 1APS 0029  AP
Produced by (code/name) : 150 /
Produced for            : PETROBRAS
Created                 : 2019-11-11 17:04:41

Created by              : PETROBANK, (version: 4)
Other programs/services : XFER_DLWRIT 1.00


In [ ]:
origin3, *origin_tail3 = lista_logicos2[1].origins
origin3.describe()

------
Origin
------
name   : DEFINING-ORIGIN
origin : 0
copy   : 0

Logical file ID          : 1APS 0029  APBRA1APS 0029  AP3
File set name and number : SPEP-AP-01C1 / 37990
File number and type     : 169414237 / RECALL-DATA

Field                   : LIS_2000/11/11:17:53:
Well (id/name)          : 1APS 0029  AP / 1APS 0029  AP
Produced by (code/name) : 150 /
Produced for            : PETROBRAS
Created                 : 2019-11-11 17:04:41

Created by              : PETROBANK, (version: 4)
Other programs/services : XFER_DLWRIT 1.00


In [ ]:
#A partir dessa quarto lógico, o cabeçalho é o mesmo.
origin4, *origin_tail4 = lista_logicos2[2].origins
origin4.describe()

------
Origin
------
name   : DEFINING-ORIGIN
origin : 0
copy   : 0

Logical file ID          : 1APS 0029  APBRA1APS 0029  AP4
File set name and number : SPEP-AP-01C1 / 37990
File number and type     : 169414237 / RECALL-DATA

Field                   : LIS_2000/11/11:17:53:
Well (id/name)          : 1APS 0029  AP / 1APS 0029  AP
Produced by (code/name) : 150 /
Produced for            : PETROBRAS
Created                 : 2019-11-11 17:04:41

Created by              : PETROBANK, (version: 4)
Other programs/services : XFER_DLWRIT 1.00


### EXTRAINDO OS FRAMES DOS ARQUIVOS LÓGICOS

In [62]:
lista_frames = []
for i in lista_logicos2:
    for fr in i.frames:
        lista_frames.append(fr)

In [63]:
#manipulação para remover os frames duplicados sem alterar o type
df_frames = pd.DataFrame(lista_frames, index = lista_frames)
df_frames.index = df_frames.index.astype('string')
df_frames = df_frames[~df_frames.index.duplicated(keep = 'first')]

In [64]:
lista_frames2 = df_frames[0].to_list()

In [65]:
lista_frames2

[Frame([0,1,BHC/GR/BOREHOLE-DEPTH/1/1.0/M]),
 Frame([0,1,CAL/BOREHOLE-DEPTH/1/1.0/M]),
 Frame([0,1,IES/BOREHOLE-DEPTH/1/.20/M]),
 Frame([0,1,HDT/BOREHOLE-DEPTH/1/1.0/M]),
 Frame([0,1,FDC/CNL/CNL/BOREHOLE-DEPTH/1/1.0/M]),
 Frame([0,1,GR/BOREHOLE-DEPTH/1/1.0/M]),
 Frame([0,1,HDT/HDT/BOREHOLE-DEPTH/1/1.0/M]),
 Frame([0,1,FDC/CNL/BOREHOLE-DEPTH/1/1.0/M]),
 Frame([0,1,FDC/CNL/BOREHOLE-DEPTH/1/.20/M]),
 Frame([0,1,BSL/GR/CAL/BOREHOLE-DEPTH/1/.20/M]),
 Frame([0,1,BSL/GR/BOREHOLE-DEPTH/1/.20/M]),
 Frame([0,1,INTEGRACAO_DO_SONICO/BOREHOLE-DEPTH/1/.20/M]),
 Frame([0,1,TTI/BOREHOLE-DEPTH/1/.20/M])]

### FILTRANDO OS CANAIS NECESSÁRIOS 

Como a quantidade de curvas dentro dos arquivos dlis é muito grande, para diminuir a demanda computacional, é necessário filtrar as curvas que realmente serão utilizadas no projeto.

#### APLICANDO A FUNÇÃO SUMMARY_DATAFRAME E CRIANDO UM DATAFRAME COM OS PARÂMETROS DOS CANAIS: LONG NAME, DIMENSION, UNITS, FRAME.

A partir do parâmetro "Long Name", poderemos filtrar os nomes dos canais pelo parâmetro físico que ele representa.

In [71]:
#aplicando a função summary dataframe
lista_sumario = []
for i in lista_logicos2:
    lista_sumario.append(summary_dataframe(i.channels, name='Name', long_name='Long Name', dimension='Dimension', units='Units', frame='Frame'))
    

In [72]:
#concatenando todos os dataframes em somente um
channels = pd.concat(lista_sumario)

#### FILTRANDO OS CANAIS.

Primeiramente, colocamos o nome da(s) propriedade(s) escolhida(s) na lista abaixo. Esse nome pode ser conferido no dataframe acima (Ctrl+f para verificar ortografia, letras maiúsculas e minúsculas, etc.)

In [73]:
#lista que deve conter os nomes das propriedades físicas que o usuário deseja salvar
propriedades = ['Gamma']

Após isso, utilizamos a função filtra_canais para filtrarmos os nomes dos canais que contém a propriedade escolhida e colocamos os resultados na lista_alvo.

In [74]:
lista_alvo = []
for i in propriedades:
    #filra_canais(nome do dataframe com os parâmetros definida acima, i = cada propriedade da lista "propriedades")
    lista_alvo.append(filtro(channels, i))

In [75]:
#utilizando a função para desencadear a lista
lista_alvo =  list(flatten(lista_alvo))

In [76]:
#utilizando a função para remover os nomes repetidos
lista_alvo = remove_repetidos(sorted(lista_alvo))

In [77]:
lista_alvo

['GR']

#### FILTRANDO A PROFUNDIDADE

A variável profundidade pode vir com diversos nomes, como: TDEP, DEPTH, INDEX444, etc. Por isso, criamos uma lista que contém uma parte do Long Name (do mesmo modo que na filtragem de canais). Somente analisando o dataframe 'channels', conseguimos saber qual é a variável de profundidade.

In [78]:
#no caso da Bacia do Amazonas, a string 'INDEX' aparece nos long names das profundidades.
profundidade = ['INDEX']

In [79]:
lista_profundidade = []
for i in profundidade:
    #filra_canais(nome do dataframe com os parâmetros definida acima, i = cada propriedade da lista "propriedades")
    lista_profundidade.append(filtro(channels, i))

In [80]:
#aplicando função para desencadear a lista
lista_profundidade = list(flatten(lista_profundidade))

In [ ]:
#Caso haja o canal "TDEP", descomente o código abaixo e utilize-o para ser a lista_profundidade
#lista_profundidade = ['TDEP']

In [81]:
#aplicando a função para remover os repetidos
lista_profundidade= remove_repetidos(sorted(lista_profundidade))

In [82]:
lista_profundidade

['INDEX440',
 'INDEX441',
 'INDEX442',
 'INDEX443',
 'INDEX444',
 'INDEX445',
 'INDEX446',
 'INDEX447',
 'INDEX448',
 'INDEX449',
 'INDEX450']

**OBS2.: QUANDO A PROFUNDIDADE FOR 'TDEP', ELA GERALMENTE VEM EM PÉS. A CONVERSÃO DE PÉS PARA METROS É MULTIPLICAR O VALOR POR 0.00254 . ESSA MULTIPLICAÇÃO DEVERÁ SER COLOCADA NO CÓDIGO DE TRANFORMAR CURVAS EM DATAFRAMES ABAIXO (ESTÁ SINALIZADO NA CÉLULA).**

### FUNÇÃO QUE TRANSFORMA OS VALORES DOS CANAIS EM DATAFRAME DE ACORDO COM A PROFUNDIDADE.

Para aplicarmos a função, utilizaremos 4 listas: lista_logicos2, lista_frames2, lista_alvo e lista_profundidade.

In [83]:
lista_curvas = []


for i in lista_logicos2:
    
    for fr in lista_frames2:
        
        try:

            #pegando as curvas de todos os frames
            curves = fr.curves()
            #curves = frame.curves()

        except Exception as err: 

            print(f' *O arquivo {i}, frame {fr} possui o erro: {err}*')
            pass

        for v in lista_alvo:

            curve_name = str(v) + '_' + str(i) + '_' + str(fr) 

            for profundidade in lista_profundidade:    

                try:

                    curvas = (pd.DataFrame(curves[v], columns = [curve_name], index = curves[profundidade]))
                    
                    #CASO A PROFUNDIDADE ESTEJA EM PÉS, UTILIZE:
                    #lista_curvas.append(pd.DataFrame(curves[v], columns = [curva_name], index = curves[profundidade]*0.00254))
                    
                    #transformando o valor -999.25 em nulo
                    curvas[curve_name][curvas[curve_name] == -999.250000] = np.nan
                    
                    lista_curvas.append(curvas)
                    
                    #para alertar quais lógicos possuem as curvas
                    print(f'** O arquivo {i}, frame {fr}, possui a curva {v}')


                except Exception as e: 

                    print(f' O arquivo {i}, frame {fr}, possui o erro: {e}')
                    pass

 O arquivo LogicalFile(1APS 0029  APBRA1APS 0029  AP2), frame Frame([0,1,BHC/GR/BOREHOLE-DEPTH/1/1.0/M]), possui o erro: no field of name GR
 O arquivo LogicalFile(1APS 0029  APBRA1APS 0029  AP2), frame Frame([0,1,BHC/GR/BOREHOLE-DEPTH/1/1.0/M]), possui o erro: no field of name GR
 O arquivo LogicalFile(1APS 0029  APBRA1APS 0029  AP2), frame Frame([0,1,BHC/GR/BOREHOLE-DEPTH/1/1.0/M]), possui o erro: no field of name GR
 O arquivo LogicalFile(1APS 0029  APBRA1APS 0029  AP2), frame Frame([0,1,BHC/GR/BOREHOLE-DEPTH/1/1.0/M]), possui o erro: no field of name GR
 O arquivo LogicalFile(1APS 0029  APBRA1APS 0029  AP2), frame Frame([0,1,BHC/GR/BOREHOLE-DEPTH/1/1.0/M]), possui o erro: no field of name GR
 O arquivo LogicalFile(1APS 0029  APBRA1APS 0029  AP2), frame Frame([0,1,BHC/GR/BOREHOLE-DEPTH/1/1.0/M]), possui o erro: no field of name GR
 O arquivo LogicalFile(1APS 0029  APBRA1APS 0029  AP2), frame Frame([0,1,BHC/GR/BOREHOLE-DEPTH/1/1.0/M]), possui o erro: no field of name GR
 O arquivo Lo

 O arquivo LogicalFile(1APS 0029  APBRA1APS 0029  AP3), frame Frame([0,1,BSL/GR/BOREHOLE-DEPTH/1/.20/M]), possui o erro: no field of name INDEX440
 O arquivo LogicalFile(1APS 0029  APBRA1APS 0029  AP3), frame Frame([0,1,BSL/GR/BOREHOLE-DEPTH/1/.20/M]), possui o erro: no field of name INDEX441
 O arquivo LogicalFile(1APS 0029  APBRA1APS 0029  AP3), frame Frame([0,1,BSL/GR/BOREHOLE-DEPTH/1/.20/M]), possui o erro: no field of name INDEX442
 O arquivo LogicalFile(1APS 0029  APBRA1APS 0029  AP3), frame Frame([0,1,BSL/GR/BOREHOLE-DEPTH/1/.20/M]), possui o erro: no field of name INDEX443
 O arquivo LogicalFile(1APS 0029  APBRA1APS 0029  AP3), frame Frame([0,1,BSL/GR/BOREHOLE-DEPTH/1/.20/M]), possui o erro: no field of name INDEX444
 O arquivo LogicalFile(1APS 0029  APBRA1APS 0029  AP3), frame Frame([0,1,BSL/GR/BOREHOLE-DEPTH/1/.20/M]), possui o erro: no field of name INDEX445
 O arquivo LogicalFile(1APS 0029  APBRA1APS 0029  AP3), frame Frame([0,1,BSL/GR/BOREHOLE-DEPTH/1/.20/M]), possui o err

** O arquivo LogicalFile(1APS 0029  APBRA1APS 0029  AP4), frame Frame([0,1,BSL/GR/BOREHOLE-DEPTH/1/.20/M]), possui a curva GR
 O arquivo LogicalFile(1APS 0029  APBRA1APS 0029  AP4), frame Frame([0,1,BSL/GR/BOREHOLE-DEPTH/1/.20/M]), possui o erro: no field of name INDEX448
 O arquivo LogicalFile(1APS 0029  APBRA1APS 0029  AP4), frame Frame([0,1,BSL/GR/BOREHOLE-DEPTH/1/.20/M]), possui o erro: no field of name INDEX449
 O arquivo LogicalFile(1APS 0029  APBRA1APS 0029  AP4), frame Frame([0,1,BSL/GR/BOREHOLE-DEPTH/1/.20/M]), possui o erro: no field of name INDEX450
 O arquivo LogicalFile(1APS 0029  APBRA1APS 0029  AP4), frame Frame([0,1,INTEGRACAO_DO_SONICO/BOREHOLE-DEPTH/1/.20/M]), possui o erro: no field of name GR
 O arquivo LogicalFile(1APS 0029  APBRA1APS 0029  AP4), frame Frame([0,1,INTEGRACAO_DO_SONICO/BOREHOLE-DEPTH/1/.20/M]), possui o erro: no field of name GR
 O arquivo LogicalFile(1APS 0029  APBRA1APS 0029  AP4), frame Frame([0,1,INTEGRACAO_DO_SONICO/BOREHOLE-DEPTH/1/.20/M]), pos

 O arquivo LogicalFile(1APS 0029  APBRA1APS 0029  AP5), frame Frame([0,1,TTI/BOREHOLE-DEPTH/1/.20/M]), possui o erro: no field of name GR
 O arquivo LogicalFile(1APS 0029  APBRA1APS 0029  AP5), frame Frame([0,1,TTI/BOREHOLE-DEPTH/1/.20/M]), possui o erro: no field of name GR
 O arquivo LogicalFile(1APS 0029  APBRA1APS 0029  AP5), frame Frame([0,1,TTI/BOREHOLE-DEPTH/1/.20/M]), possui o erro: no field of name GR
 O arquivo LogicalFile(1APS 0029  APBRA1APS 0029  AP5), frame Frame([0,1,TTI/BOREHOLE-DEPTH/1/.20/M]), possui o erro: no field of name GR
 O arquivo LogicalFile(1APS 0029  APBRA1APS 0029  AP5), frame Frame([0,1,TTI/BOREHOLE-DEPTH/1/.20/M]), possui o erro: no field of name GR
 O arquivo LogicalFile(1APS 0029  APBRA1APS 0029  AP5), frame Frame([0,1,TTI/BOREHOLE-DEPTH/1/.20/M]), possui o erro: no field of name GR
 O arquivo LogicalFile(1APS 0029  APBRA1APS 0029  AP5), frame Frame([0,1,TTI/BOREHOLE-DEPTH/1/.20/M]), possui o erro: no field of name GR
 O arquivo LogicalFile(1APS 0029  

 O arquivo LogicalFile(1APS 0029  APBRA1APS 0029  AP7), frame Frame([0,1,FDC/CNL/BOREHOLE-DEPTH/1/.20/M]), possui o erro: no field of name GR
 O arquivo LogicalFile(1APS 0029  APBRA1APS 0029  AP7), frame Frame([0,1,FDC/CNL/BOREHOLE-DEPTH/1/.20/M]), possui o erro: no field of name GR
 O arquivo LogicalFile(1APS 0029  APBRA1APS 0029  AP7), frame Frame([0,1,FDC/CNL/BOREHOLE-DEPTH/1/.20/M]), possui o erro: no field of name GR
 O arquivo LogicalFile(1APS 0029  APBRA1APS 0029  AP7), frame Frame([0,1,FDC/CNL/BOREHOLE-DEPTH/1/.20/M]), possui o erro: no field of name GR
 O arquivo LogicalFile(1APS 0029  APBRA1APS 0029  AP7), frame Frame([0,1,FDC/CNL/BOREHOLE-DEPTH/1/.20/M]), possui o erro: no field of name GR
 O arquivo LogicalFile(1APS 0029  APBRA1APS 0029  AP7), frame Frame([0,1,FDC/CNL/BOREHOLE-DEPTH/1/.20/M]), possui o erro: no field of name GR
 O arquivo LogicalFile(1APS 0029  APBRA1APS 0029  AP7), frame Frame([0,1,FDC/CNL/BOREHOLE-DEPTH/1/.20/M]), possui o erro: no field of name GR
 O arq

...

 O arquivo LogicalFile(1APS 0029  APBRA1APS 0029  AP22), frame Frame([0,1,GR/BOREHOLE-DEPTH/1/1.0/M]), possui o erro: no field of name GR
 O arquivo LogicalFile(1APS 0029  APBRA1APS 0029  AP22), frame Frame([0,1,GR/BOREHOLE-DEPTH/1/1.0/M]), possui o erro: no field of name GR
 O arquivo LogicalFile(1APS 0029  APBRA1APS 0029  AP22), frame Frame([0,1,GR/BOREHOLE-DEPTH/1/1.0/M]), possui o erro: no field of name GR
 O arquivo LogicalFile(1APS 0029  APBRA1APS 0029  AP22), frame Frame([0,1,GR/BOREHOLE-DEPTH/1/1.0/M]), possui o erro: no field of name GR
 O arquivo LogicalFile(1APS 0029  APBRA1APS 0029  AP22), frame Frame([0,1,GR/BOREHOLE-DEPTH/1/1.0/M]), possui o erro: no field of name GR
 O arquivo LogicalFile(1APS 0029  APBRA1APS 0029  AP22), frame Frame([0,1,GR/BOREHOLE-DEPTH/1/1.0/M]), possui o erro: no field of name GR
 O arquivo LogicalFile(1APS 0029  APBRA1APS 0029  AP22), frame Frame([0,1,GR/BOREHOLE-DEPTH/1/1.0/M]), possui o erro: no field of name GR
 O arquivo LogicalFile(1APS 0029  

 O arquivo LogicalFile(1APS 0029  APBRA1APS 0029  AP23), frame Frame([0,1,IES/BOREHOLE-DEPTH/1/.20/M]), possui o erro: no field of name GR
 O arquivo LogicalFile(1APS 0029  APBRA1APS 0029  AP23), frame Frame([0,1,IES/BOREHOLE-DEPTH/1/.20/M]), possui o erro: no field of name GR
 O arquivo LogicalFile(1APS 0029  APBRA1APS 0029  AP23), frame Frame([0,1,IES/BOREHOLE-DEPTH/1/.20/M]), possui o erro: no field of name GR
 O arquivo LogicalFile(1APS 0029  APBRA1APS 0029  AP23), frame Frame([0,1,IES/BOREHOLE-DEPTH/1/.20/M]), possui o erro: no field of name GR
 O arquivo LogicalFile(1APS 0029  APBRA1APS 0029  AP23), frame Frame([0,1,IES/BOREHOLE-DEPTH/1/.20/M]), possui o erro: no field of name GR
 O arquivo LogicalFile(1APS 0029  APBRA1APS 0029  AP23), frame Frame([0,1,IES/BOREHOLE-DEPTH/1/.20/M]), possui o erro: no field of name GR
 O arquivo LogicalFile(1APS 0029  APBRA1APS 0029  AP23), frame Frame([0,1,IES/BOREHOLE-DEPTH/1/.20/M]), possui o erro: no field of name GR
 O arquivo LogicalFile(1APS

 O arquivo LogicalFile(1APS 0029  APBRA1APS 0029  AP24), frame Frame([0,1,FDC/CNL/BOREHOLE-DEPTH/1/.20/M]), possui o erro: no field of name GR
 O arquivo LogicalFile(1APS 0029  APBRA1APS 0029  AP24), frame Frame([0,1,FDC/CNL/BOREHOLE-DEPTH/1/.20/M]), possui o erro: no field of name GR
 O arquivo LogicalFile(1APS 0029  APBRA1APS 0029  AP24), frame Frame([0,1,FDC/CNL/BOREHOLE-DEPTH/1/.20/M]), possui o erro: no field of name GR
 O arquivo LogicalFile(1APS 0029  APBRA1APS 0029  AP24), frame Frame([0,1,FDC/CNL/BOREHOLE-DEPTH/1/.20/M]), possui o erro: no field of name GR
 O arquivo LogicalFile(1APS 0029  APBRA1APS 0029  AP24), frame Frame([0,1,FDC/CNL/BOREHOLE-DEPTH/1/.20/M]), possui o erro: no field of name GR
 O arquivo LogicalFile(1APS 0029  APBRA1APS 0029  AP24), frame Frame([0,1,FDC/CNL/BOREHOLE-DEPTH/1/.20/M]), possui o erro: no field of name GR
 O arquivo LogicalFile(1APS 0029  APBRA1APS 0029  AP24), frame Frame([0,1,FDC/CNL/BOREHOLE-DEPTH/1/.20/M]), possui o erro: no field of name GR

** O arquivo LogicalFile(1APS 0029  APBRA1APS 0029  AP25), frame Frame([0,1,BSL/GR/CAL/BOREHOLE-DEPTH/1/.20/M]), possui a curva GR
 O arquivo LogicalFile(1APS 0029  APBRA1APS 0029  AP25), frame Frame([0,1,BSL/GR/CAL/BOREHOLE-DEPTH/1/.20/M]), possui o erro: no field of name INDEX446
 O arquivo LogicalFile(1APS 0029  APBRA1APS 0029  AP25), frame Frame([0,1,BSL/GR/CAL/BOREHOLE-DEPTH/1/.20/M]), possui o erro: no field of name INDEX447
 O arquivo LogicalFile(1APS 0029  APBRA1APS 0029  AP25), frame Frame([0,1,BSL/GR/CAL/BOREHOLE-DEPTH/1/.20/M]), possui o erro: no field of name INDEX448
 O arquivo LogicalFile(1APS 0029  APBRA1APS 0029  AP25), frame Frame([0,1,BSL/GR/CAL/BOREHOLE-DEPTH/1/.20/M]), possui o erro: no field of name INDEX449
 O arquivo LogicalFile(1APS 0029  APBRA1APS 0029  AP25), frame Frame([0,1,BSL/GR/CAL/BOREHOLE-DEPTH/1/.20/M]), possui o erro: no field of name INDEX450
 O arquivo LogicalFile(1APS 0029  APBRA1APS 0029  AP25), frame Frame([0,1,BSL/GR/BOREHOLE-DEPTH/1/.20/M]), po

 O arquivo LogicalFile(1APS 0029  APBRA1APS 0029  AP1), frame Frame([0,1,FDC/CNL/BOREHOLE-DEPTH/1/.20/M]), possui o erro: no field of name GR
 O arquivo LogicalFile(1APS 0029  APBRA1APS 0029  AP1), frame Frame([0,1,FDC/CNL/BOREHOLE-DEPTH/1/.20/M]), possui o erro: no field of name GR
 O arquivo LogicalFile(1APS 0029  APBRA1APS 0029  AP1), frame Frame([0,1,FDC/CNL/BOREHOLE-DEPTH/1/.20/M]), possui o erro: no field of name GR
 O arquivo LogicalFile(1APS 0029  APBRA1APS 0029  AP1), frame Frame([0,1,FDC/CNL/BOREHOLE-DEPTH/1/.20/M]), possui o erro: no field of name GR
 O arquivo LogicalFile(1APS 0029  APBRA1APS 0029  AP1), frame Frame([0,1,FDC/CNL/BOREHOLE-DEPTH/1/.20/M]), possui o erro: no field of name GR
 O arquivo LogicalFile(1APS 0029  APBRA1APS 0029  AP1), frame Frame([0,1,FDC/CNL/BOREHOLE-DEPTH/1/.20/M]), possui o erro: no field of name GR
 O arquivo LogicalFile(1APS 0029  APBRA1APS 0029  AP1), frame Frame([0,1,FDC/CNL/BOREHOLE-DEPTH/1/.20/M]), possui o erro: no field of name GR
 O arq

In [84]:
#removendo os valores constantes (erro comum no dado)
lista_curvas2 = []
for i in lista_curvas:
    lista_curvas2.append(i.reset_index().drop_duplicates(subset='index', keep='last').set_index('index'))

In [85]:
lista_curvas2

[        GR_LogicalFile(1APS 0029  APBRA1APS 0029  AP2)_Frame([0,1,BSL/GR/CAL/BOREHOLE-DEPTH/1/.20/M])
 index                                                                                                
 3656.0                                          75.466888                                            
 3656.2                                          72.092484                                            
 3656.4                                          24.713242                                            
 3656.6                                          22.932465                                            
 3656.8                                          21.986649                                            
 ...                                                   ...                                            
 4382.2                                                NaN                                            
 4382.4                                                NaN               

In [86]:
curvas_alvo = pd.concat(lista_curvas2, axis = 1)

In [87]:
#resetando o index e ordenando os valores em profundidade crescente 
curvas = curvas_alvo.reset_index().sort_values('index')

In [88]:
curvas

,index,"GR_LogicalFile(1APS 0029 APBRA1APS 0029 AP2)_Frame([0,1,BSL/GR/CAL/BOREHOLE-DEPTH/1/.20/M])","GR_LogicalFile(1APS 0029 APBRA1APS 0029 AP2)_Frame([0,1,BSL/GR/BOREHOLE-DEPTH/1/.20/M])","GR_LogicalFile(1APS 0029 APBRA1APS 0029 AP3)_Frame([0,1,BSL/GR/CAL/BOREHOLE-DEPTH/1/.20/M])","GR_LogicalFile(1APS 0029 APBRA1APS 0029 AP3)_Frame([0,1,BSL/GR/BOREHOLE-DEPTH/1/.20/M])","GR_LogicalFile(1APS 0029 APBRA1APS 0029 AP4)_Frame([0,1,BSL/GR/CAL/BOREHOLE-DEPTH/1/.20/M])","GR_LogicalFile(1APS 0029 APBRA1APS 0029 AP4)_Frame([0,1,BSL/GR/BOREHOLE-DEPTH/1/.20/M])","GR_LogicalFile(1APS 0029 APBRA1APS 0029 AP5)_Frame([0,1,BSL/GR/CAL/BOREHOLE-DEPTH/1/.20/M])","GR_LogicalFile(1APS 0029 APBRA1APS 0029 AP5)_Frame([0,1,BSL/GR/BOREHOLE-DEPTH/1/.20/M])","GR_LogicalFile(1APS 0029 APBRA1APS 0029 AP6)_Frame([0,1,BSL/GR/CAL/BOREHOLE-DEPTH/1/.20/M])",...,"GR_LogicalFile(1APS 0029 APBRA1APS 0029 AP22)_Frame([0,1,BSL/GR/CAL/BOREHOLE-DEPTH/1/.20/M])","GR_LogicalFile(1APS 0029 APBRA1APS 0029 AP22)_Frame([0,1,BSL/GR/BOREHOLE-DEPTH/1/.20/M])","GR_LogicalFile(1APS 0029 APBRA1APS 0029 AP23)_Frame([0,1,BSL/GR/CAL/BOREHOLE-DEPTH/1/.20/M])","GR_LogicalFile(1APS 0029 APBRA1APS 0029 AP23)_Frame([0,1,BSL/GR/BOREHOLE-DEPTH/1/.20/M])","GR_LogicalFile(1APS 0029 APBRA1APS 0029 AP24)_Frame([0,1,BSL/GR/CAL/BOREHOLE-DEPTH/1/.20/M])","GR_LogicalFile(1APS 0029 APBRA1APS 0029 AP24)_Frame([0,1,BSL/GR/BOREHOLE-DEPTH/1/.20/M])","GR_LogicalFile(1APS 0029 APBRA1APS 0029 AP25)_Frame([0,1,BSL/GR/CAL/BOREHOLE-DEPTH/1/.20/M])","GR_LogicalFile(1APS 0029 APBRA1APS 0029 AP25)_Frame([0,1,BSL/GR/BOREHOLE-DEPTH/1/.20/M])","GR_LogicalFile(1APS 0029 APBRA1APS 0029 AP1)_Frame([0,1,BSL/GR/CAL/BOREHOLE-DEPTH/1/.20/M])","GR_LogicalFile(1APS 0029 APBRA1APS 0029 AP1)_Frame([0,1,BSL/GR/BOREHOLE-DEPTH/1/.20/M])"
3636,120.0,NaN,9.759691,NaN,9.759691,NaN,9.759691,NaN,9.759691,NaN,...,NaN,9.759691,NaN,9.759691,NaN,9.759691,NaN,9.759691,NaN,9.759691
3637,120.2,NaN,10.393160,NaN,10.393160,NaN,10.393160,NaN,10.393160,NaN,...,NaN,10.393160,NaN,10.393160,NaN,10.393160,NaN,10.393160,NaN,10.393160
3638,120.4,NaN,10.642555,NaN,10.642555,NaN,10.642555,NaN,10.642555,NaN,...,NaN,10.642555,NaN,10.642555,NaN,10.642555,NaN,10.642555,NaN,10.642555
3639,120.6,NaN,9.729208,NaN,9.729208,NaN,9.729208,NaN,9.729208,NaN,...,NaN,9.729208,NaN,9.729208,NaN,9.729208,NaN,9.729208,NaN,9.729208
3640,120.8,NaN,8.981640,NaN,8.981640,NaN,8.981640,NaN,8.981640,NaN,...,NaN,8.981640,NaN,8.981640,NaN,8.981640,NaN,8.981640,NaN,8.981640
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3631,4382.2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3632,4382.4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3633,4382.6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3634,4382.8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## ACOPLANDO A LITOLOGIA

#### LENDO OS ARQUIVOS AGPS 

In [90]:
path = 'agps'

In [91]:
lista_agp = []

#importando todos os arquivos dlis da pasta '
for i in (glob.glob(f'{path}**/*txt')):
    lista_agp.append(i)

In [92]:
lista_agp

['agps\\1-BRSA-18-ESS_dados.txt',
 'agps\\1-BRSA-186-ESS_dados.txt',
 'agps\\1APS0029AP_agp.txt',
 'agps\\1CES0112CE_agp.txt']

#### É IMPORTANTE QUE O NOME DAS COLUNAS SEJAM SEMPRE ('Profundidade', 'Code', 'Rock'), POIS A FUNÇÃO PARA  ACOPLAR POSSUI CONDICIONAIS QUE DEPENDEM DESSES NOMES

In [100]:
lito = pd.read_csv(lista_agp[2], sep='\s+', usecols=(0,2,3), index_col=False, na_values= ' ', skiprows=1, names=('Profundidade', 'Code', 'Rock'), encoding = "ISO-8859-1" ) 

In [101]:
lito

,Profundidade,Code,Rock
0,IDENTIFICADOR,NB22W33NE34M55A,BACIA
1,CAMPO,DATUM,:
2,LATITUDE,4.20613,(
3,MERID.CENTRAL,-,51
4,DISTRITO,E&P-AM,ESTADO
...,...,...,...
673,MODO,DISSEMINACAO,DO
674,AUREOLAR,NaN,NaN
675,RADIAL,NaN,NaN
676,NAO,APLICA,N


#### FILTRANDO A PARTE DO DATAFRAME QUE CONTÉM AS INFORMAÇÕES NECESSÁRIAS: LITOLOGIA, CODE E ROCK

In [102]:
litologia = lito.iloc[243:586]

In [103]:
curvas_lito = acoplador(curvas, litologia)

O DATAFRAME POSSUI AS COLUNAS CODE E ROCK


## PROBLEMA: ARQUIVOS AGP NÃO TÊM PADRÃO

In [ ]:
lista_agp

['agps\\1-BRSA-18-ESS_dados.txt',
 'agps\\1-BRSA-186-ESS_dados.txt',
 'agps\\1APS0029AP_agp.txt',
 'agps\\1CES0112CE_agp.txt']

#### EXEMPLO (DADO UTILIZADO SOMENTE PARA EXEMPLO, O ACOPLAMENTO NÃO CONDIZ COM O DLIS CORRETO): NOTE QUE PARA LER O AGP \1-BRSA-18-ESS_dados.txt, PRECISAMOS MUDAR OS PARÂMETROS DA FUNÇÃO:

In [111]:
agp1 = pd.read_csv(lista_agp[0], sep='\s+', usecols=(0,1,2), index_col=False, na_values= ' ', skiprows=1, names=('Profundidade', 'base', 'Rock'), encoding = "ISO-8859-1" )

ESSA ALTERAÇÃO SERÁ MANUAL POR ENQUANTO, POIS AINDA NÃO ACHAMOS UMA FORMA DE AUTOMATIZAR A LEITURA DESSES DADOS, PORQUE ELES NÃO SÃO PADRONIZADOS.

In [113]:
lito_agp1 = agp1.loc[1138:1800]

In [114]:
lito_agp1

,Profundidade,base,Rock
1138,2267.00/(-2243.00),3033.00/(-3008.70),FOLHELHO
1139,3033.00/(-3008.70),3034.65/(-3010.35),MARGA
1140,3034.65/(-3010.35),3040.20/(-3015.89),FOLHELHO
1141,3040.20/(-3015.89),3042.00/(-3017.69),MARGA
1142,3042.00/(-3017.69),3046.65/(-3022.34),FOLHELHO
...,...,...,...
1796,5437.20/(-5410.03),5437.65/(-5410.48),CALCILUTITO
1797,5437.65/(-5410.48),5440.05/(-5412.88),CALCARENITO
1798,5440.05/(-5412.88),5446.05/(-5418.87),CALCILUTITO
1799,5446.05/(-5418.87),5501.10/(-5473.82),ANIDRITA


In [118]:
#NO CASO DESSE AGP, PRECISAMOS FAZER TAMBÉM UMA MANIPULAÇAO DE STRING PARA PEGAR SOMENTE O VALOR DA PROFUNDIDADE
lito_agp1['Profundidade'] = lito_agp1['Profundidade'].apply(lambda x: x.split('/')[0])

In [119]:
curvas_agp1_lito = acoplador(curvas, lito_agp1)

O DATAFRAME POSSUI SOMENTE A COLUNA ROCK


In [120]:
curvas_agp1_lito

,index,"GR_LogicalFile(1APS 0029 APBRA1APS 0029 AP2)_Frame([0,1,BSL/GR/CAL/BOREHOLE-DEPTH/1/.20/M])","GR_LogicalFile(1APS 0029 APBRA1APS 0029 AP2)_Frame([0,1,BSL/GR/BOREHOLE-DEPTH/1/.20/M])","GR_LogicalFile(1APS 0029 APBRA1APS 0029 AP3)_Frame([0,1,BSL/GR/CAL/BOREHOLE-DEPTH/1/.20/M])","GR_LogicalFile(1APS 0029 APBRA1APS 0029 AP3)_Frame([0,1,BSL/GR/BOREHOLE-DEPTH/1/.20/M])","GR_LogicalFile(1APS 0029 APBRA1APS 0029 AP4)_Frame([0,1,BSL/GR/CAL/BOREHOLE-DEPTH/1/.20/M])","GR_LogicalFile(1APS 0029 APBRA1APS 0029 AP4)_Frame([0,1,BSL/GR/BOREHOLE-DEPTH/1/.20/M])","GR_LogicalFile(1APS 0029 APBRA1APS 0029 AP5)_Frame([0,1,BSL/GR/CAL/BOREHOLE-DEPTH/1/.20/M])","GR_LogicalFile(1APS 0029 APBRA1APS 0029 AP5)_Frame([0,1,BSL/GR/BOREHOLE-DEPTH/1/.20/M])","GR_LogicalFile(1APS 0029 APBRA1APS 0029 AP6)_Frame([0,1,BSL/GR/CAL/BOREHOLE-DEPTH/1/.20/M])",...,"GR_LogicalFile(1APS 0029 APBRA1APS 0029 AP23)_Frame([0,1,BSL/GR/CAL/BOREHOLE-DEPTH/1/.20/M])","GR_LogicalFile(1APS 0029 APBRA1APS 0029 AP23)_Frame([0,1,BSL/GR/BOREHOLE-DEPTH/1/.20/M])","GR_LogicalFile(1APS 0029 APBRA1APS 0029 AP24)_Frame([0,1,BSL/GR/CAL/BOREHOLE-DEPTH/1/.20/M])","GR_LogicalFile(1APS 0029 APBRA1APS 0029 AP24)_Frame([0,1,BSL/GR/BOREHOLE-DEPTH/1/.20/M])","GR_LogicalFile(1APS 0029 APBRA1APS 0029 AP25)_Frame([0,1,BSL/GR/CAL/BOREHOLE-DEPTH/1/.20/M])","GR_LogicalFile(1APS 0029 APBRA1APS 0029 AP25)_Frame([0,1,BSL/GR/BOREHOLE-DEPTH/1/.20/M])","GR_LogicalFile(1APS 0029 APBRA1APS 0029 AP1)_Frame([0,1,BSL/GR/CAL/BOREHOLE-DEPTH/1/.20/M])","GR_LogicalFile(1APS 0029 APBRA1APS 0029 AP1)_Frame([0,1,BSL/GR/BOREHOLE-DEPTH/1/.20/M])",lito,codigo_lito
3636,120.0,NaN,9.759691,NaN,9.759691,NaN,9.759691,NaN,9.759691,NaN,...,NaN,9.759691,NaN,9.759691,NaN,9.759691,NaN,9.759691,None,NaN
3637,120.2,NaN,10.393160,NaN,10.393160,NaN,10.393160,NaN,10.393160,NaN,...,NaN,10.393160,NaN,10.393160,NaN,10.393160,NaN,10.393160,None,NaN
3638,120.4,NaN,10.642555,NaN,10.642555,NaN,10.642555,NaN,10.642555,NaN,...,NaN,10.642555,NaN,10.642555,NaN,10.642555,NaN,10.642555,None,NaN
3639,120.6,NaN,9.729208,NaN,9.729208,NaN,9.729208,NaN,9.729208,NaN,...,NaN,9.729208,NaN,9.729208,NaN,9.729208,NaN,9.729208,None,NaN
3640,120.8,NaN,8.981640,NaN,8.981640,NaN,8.981640,NaN,8.981640,NaN,...,NaN,8.981640,NaN,8.981640,NaN,8.981640,NaN,8.981640,None,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3631,4382.2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,FOLHELHO,NaN
3632,4382.4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,FOLHELHO,NaN
3633,4382.6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,FOLHELHO,NaN
3634,4382.8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,FOLHELHO,NaN
